# 목적
* 로스에 미분이 필요한 경우 어떻게 해야하나?
* Large margin loss 를 구현해보기 ([논문은 링크에서 볼 수 있음](https://arxiv.org/abs/1803.05598))
* 이 노트북은 커스텀화 된 로스 구현 외에도 텐서플로우2의 아래 메쏘드에 관해 배울 수 있음
  * tf.GradientTape() 를 써서 함수를 미분하면 어떤 모양이 되나 확인
  * tf.gather_nd 가 뭔 일을 하나 보기

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers, losses, metrics, optimizers
import numpy as np
import time
import IPython.display as ipd

In [3]:
BATCH_SIZE = 32
EPOCHS = 2 # 개 오래 걸려서 조금만 돌리자
NB_CLASS = 10

이런 걸 알아보기 전에 먼저 데이터랑 모델을 준비하자.

* 데이터

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = np.expand_dims(np.array(X_train, dtype=np.float32), axis=-1)
X_test = np.expand_dims(np.array(X_test, dtype=np.float32), axis=-1)

X_train /= 255.
X_test /= 255.

np.random.seed(0)
idx = np.random.randint(0, len(X_train), size=30000)
X_train = X_train[idx]
y_train = y_train[idx]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(30000, 28, 28, 1) (30000,)
(10000, 28, 28, 1) (10000,)


In [5]:
traindataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(BATCH_SIZE)
testdataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

* 모델

In [6]:
class Classifier(models.Model):
    def __init__(self):
        super(Classifier, self).__init__()
        self.c1 = layers.Conv2D(256, (3,3), activation=tf.nn.relu)
        self.c2 = layers.Conv2D(256, (3,3), activation=tf.nn.relu)
        self.flatten = layers.Flatten()
        self.latent = layers.Dense(4, activation=tf.nn.tanh)
        self.dense = layers.Dense(10, activation=tf.nn.softmax)
        
    def call(self, inputs, training=False):
        x = self.c1(inputs)
        x = self.c2(x)
        x = self.flatten(x)
        x = self.latent(x)
        x = self.dense(x)
        return x
    
classifier = Classifier()
classifier.build((None, 28, 28, 1))

## 1. tf.GradientTape()가 미분하면 무슨 모양이 되나?

* classifier 에서는 BATCH_SIZE X 10 차원의 값이 출력 됨
* 요걸 input x로 미분하면 무슨 모양? (이때 x = BATCH_SIZE X 28 X 28 X 1 차원임)
* 상식적으로 BATCH_SIZE X 10 X 28 X 28 X 1 이 될 것 같지만, 미분 된 모양은 BATCH_SIZE X 28 X 28 X 1 이 됨

In [7]:
x = tf.ones((1, 28, 28, 1), dtype=tf.float32)
print('x', x.shape)

y_class = list()
# persistent=True 를 해두면 이 tape로 몇 번이고 미분을 할 수 있다. 아니면 한 번 하면 끝
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    y = classifier(x)
    print('y', y.shape)
    
    for i in range(NB_CLASS):
        y_class.append(y[0][i])
    
grads = tape.gradient(y, x)
print('grads',grads.shape)

x (1, 28, 28, 1)
y (1, 10)
grads (1, 28, 28, 1)


* 그래서 어거지로 클래스별로 미분을 하려고 하면 위 코드처럼 with 블록 안에 클래스별로 분해된 값을 따로 저장해두어야 함

In [8]:
grad_list = list()

for i in range(NB_CLASS):
    grad_list.append(tape.gradient(y_class[i], x))
    
grad_list = np.array(grad_list)
print(grad_list.shape)

(10, 1, 28, 28, 1)


## 2. tf.gather_nd
* 인덱스 위치에 해당하는 텐서를 가져옴

**주의**
* 이름 비슷한 tf.gather가 있지만 이 놈은 텐서를 잘개 쪼개서 값을 가져오는게 아님
* 이놈을 쓰려면 params에 넣을 텐서를 이런 저런 모양 변환을 해주어야 해서 복잡하니 그냥 tf.gather_nd나 쓰자. ([링크 참고](https://stackoverflow.com/questions/36764791/in-tensorflow-how-to-use-tf-gather-for-the-last-dimension))

In [9]:
p = np.random.randint(0, 100, size=(4,3))
p

array([[56,  6, 24],
       [32, 71,  2],
       [74, 49, 65],
       [98, 66, 95]])

In [10]:
tf.gather_nd(params=p, indices=[[1,0],[0,2],[0,1],[2,2],[3,2]])

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([32, 24,  6, 65, 95])>

## Large Margin Loss
* 1, 2 두 가지를 이용해서 large margin loss 를 구현해보자.

In [11]:
loss_obj = losses.SparseCategoricalCrossentropy()
acc_obj = metrics.SparseCategoricalAccuracy()
opt = optimizers.RMSprop(1e-4)

loss = metrics.Mean()
acc = metrics.Mean()

In [12]:
l2_distance = losses.MeanSquaredError()

* 아래는 원 논문의 로스 수식을 그대로 구현 함

$$
\hat{w} \overset{\triangle}{=} \arg \min_w \sum_{\mathscr{l}, k} \mathscr{A}_{i\neq y_k} \max \bigg\{0, \gamma_\mathscr{l} + {f_i (x_k) - f_{y_k} (x_k) \over \epsilon + \lVert \nabla_{h_\mathscr{l}} f_i (x_k) - \nabla_{h_\mathscr{l}} f_{y_k} (x_k)\rVert_q } \bigg\}
$$

(원 저자들은 연산량을 줄이기 위해 정답 클래스와 가장 큰 차이나는 오답 클래스 가져다 그것만 썼지만 여기선 그런거 없음)

In [13]:
def large_margin(xk, _y):
    # 학습 시키다 보면 데이터 개수가 배치보다 작은 경우 있어서
    _BATCH_SIZE = len(xk)
    
    # 정답 인덱싱 위한 것
    yk = np.array(list(zip(np.arange(_BATCH_SIZE), _y))) # BATCH_SIZE X 2
    
    # f_i (x_k) : tape 로 미분한 모양이 우리 생각과 달라서 리스트 만들어둠. 위 1. tf.GradientTape()가 미분하면 무슨 모양이 되나?를 확인
    _fi = list()

    with tf.GradientTape(persistent=True) as tape:
        tape.watch(xk)
        h0 = classifier.layers[0](xk) # conv2d
        h1 = classifier.layers[1](h0) # conv2d_1
        _x = classifier.layers[2](h1) # flatten
        h2 = classifier.layers[3](_x) # dense
        fi = classifier.layers[4](h2) # dense_1

        for i in range(NB_CLASS):
            _fi.append(fi[:,i]) # BATCH_SIZE X 1 (클래스마다)
        # f_{y_k} (x_k),  정답 인덱싱으로부터 정답 소프트맥스 값만으로 된 f_yk (xk) 만듦
        fyk = tf.gather_nd(fi, yk) # 위 2. tf.gather_nd 설명 보자
        # 
        fyk = tf.transpose(tf.cast([fyk for _ in range(NB_CLASS)], dtype=tf.float32))
        
    # 분자
    numerator = fi - fyk

    # 분모
    _loss = get_grads(tape, numerator, _fi, yk, fyk, xk)
    _loss += get_grads(tape, numerator, _fi, yk, fyk, h0)
    _loss += get_grads(tape, numerator, _fi, yk, fyk, h1)
    _loss += get_grads(tape, numerator, _fi, yk, fyk, h2)
            
    return _loss

def get_grads(tape, numerator, _fi, yk, fyk, hi):
    # minimum distance gamma
    _gamma = .5
    _BATCH_SIZE = len(yk)
    # 분모
    Dfi = list()
    for i in range(NB_CLASS):
        # 기계적으로 계산하기 위해, 분자 0 안되도록 작은 수 더함
        Dfi.append(tape.gradient(_fi[i], hi)+1e-15)
    Dfi = tf.cast(Dfi, dtype=tf.float32)
    Dfyk = tape.gradient(fyk, hi)

    _loss = 0
    for j in range(_BATCH_SIZE):
        for i in range(NB_CLASS):
            if yk[j][1] != i: # 정답 클래스 아닌 놈들에 대해서만 로스 계산해야함
                _loss += (_gamma + tf.math.maximum(0, (numerator[j,i]) / l2_distance(Dfi[i,j], Dfyk[j]))) 
                
    return _loss

def train_step(inputs):
    xk, _y = inputs
    
    with tf.GradientTape() as tape: 
        _loss = large_margin(xk, _y)
        # acc 확인용
        pred = classifier(xk)
        
    grads = tape.gradient(_loss, classifier.trainable_variables)
    opt.apply_gradients(list(zip(grads, classifier.trainable_variables)))
    
    loss.update_state(_loss)
    acc.update_state(acc_obj(_y, pred))

In [14]:
def test_step(inputs):
    _X, _y = inputs
    
    pred = classifier(_X)
    _loss = loss_obj(_y, pred)
        
    loss.update_state(_loss)
    acc.update_state(acc_obj(_y, pred))

In [15]:
start_time = time.time()
for e in range(EPOCHS):
    epoch_time = time.time()
    for i, x in enumerate(traindataset):
        batch_time = time.time()
        train_step(x)
        ipd.clear_output(wait=True)
        print(f"{(i+1)}/{len(X_train)//BATCH_SIZE+1}, {e+1}/{EPOCHS}, loss={loss.result():.8f}, acc={100*acc.result():.2f}%, {time.time()-batch_time:.2f}secs/batch, {time.time()-epoch_time:.2f}secs/epoch, {time.time()-start_time:.2f}secs")
    loss.reset_states()
    acc.reset_states()

938/938, 2/2, loss=7750.85693359, acc=75.50%, 1.74secs/batch, 3580.28secs/epoch, 7162.37secs


In [16]:
for x in testdataset:
    test_step(x)
    
print(f"acc={100*acc.result():.2f}%")
loss.reset_states()
acc.reset_states()

acc=78.12%


## Baseline: cross entropy loss
걍 단순한 로스 쓰면?

In [32]:
cce = losses.SparseCategoricalCrossentropy()

In [33]:
def train_step(inputs):
    xk, _y = inputs
    
    with tf.GradientTape() as tape: 
        pred = classifier(xk)
        _loss = cce(_y, pred)
        
    grads = tape.gradient(_loss, classifier.trainable_variables)
    opt.apply_gradients(list(zip(grads, classifier.trainable_variables)))
    
    loss.update_state(_loss)
    acc.update_state(acc_obj(_y, pred))

In [34]:
def test_step(inputs):
    _X, _y = inputs
    
    pred = classifier(_X)
    _loss = loss_obj(_y, pred)
        
    loss.update_state(_loss)
    acc.update_state(acc_obj(_y, pred))

In [35]:
start_time = time.time()
for e in range(EPOCHS):
    epoch_time = time.time()
    for i, x in enumerate(traindataset):
        batch_time = time.time()
        train_step(x)
        ipd.clear_output(wait=True)
        print(f"{(i+1)}/{len(X_train)//BATCH_SIZE+1}, {e+1}/{EPOCHS}, loss={loss.result():.8f}, acc={100*acc.result():.2f}%, {time.time()-batch_time:.2f}secs/batch, {time.time()-epoch_time:.2f}secs/epoch, {time.time()-start_time:.2f}secs")
    loss.reset_states()
    acc.reset_states()

938/938, 2/2, loss=1.29963875, acc=68.75%, 0.01secs/batch, 14.45secs/epoch, 28.91secs


In [36]:
for x in testdataset:
    test_step(x)
    
print(f"acc={100*acc.result():.2f}%")
loss.reset_states()
acc.reset_states()

acc=70.02%


* cce: 더 빠르고 구림
* large margin loss: 느리고 좋음